In [1]:
# Install the watermark package.
# This package is used to record the versions of other packages used in this Jupyter notebook.
# https://github.com/rasbt/watermark
!pip install -q -U watermark

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from torchsummary import summary

import time
import pandas as pd
from tqdm import tqdm

In [3]:
# Load the watermark extension to display information about the Python version and installed packages.
%reload_ext watermark

# Display the versions of Python and installed packages.
%watermark -a 'Fabiano Falcão' -ws "https://fabianumfalco.github.io/" --python --iversions

Author: Fabiano Falcão

Website: https://fabianumfalco.github.io/

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.11.0

pandas     : 1.5.3
torchvision: 0.15.1
torch      : 2.0.0



In [4]:
# Check if the PyTorch CUDA library is available on the system.
# If it is available, set the device to "cuda", indicating that the GPU will be used.
# Otherwise, set the device to "cpu", indicating that the CPU will be used for computation.
# The chosen device will be used to allocate and execute PyTorch tensors.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Display the selected device
device

device(type='cuda')

In [5]:
# As for work 2, the teacher asked to use the same datasets used in work 1 and compare the results of the two works,
# set the device as CPU because I used CPU also for work 1.
device="cpu"

In [6]:
# Net class that defines the architecture of a convolutional neural network (CNN)
class NetMNIST(nn.Module):
    def __init__(self,dataset='MNIST'):
        super(NetMNIST, self).__init__()
        # Definition of the layers of the convolutional neural network
        self.conv1 = nn.Conv2d(1, 6, 5)  # First convolutional layer: input with 1 channel, 6 filters of size 5x5
        self.pool = nn.MaxPool2d(2, 2)  # Pooling layer: performs downsampling with a filter of size 2x2 and stride 2
        self.conv2 = nn.Conv2d(6, 16, 5)  # Second convolutional layer: input with 6 channels, 16 filters of size 5x5
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # First fully connected (FC) layer: 16 * 4 * 4 inputs, 120 outputs
        self.fc2 = nn.Linear(120, 84)  # Second FC layer: 120 inputs, 84 outputs
        if dataset == 'EMNIST':
            self.fc3 = nn.Linear(84, 47)  # Third FC layer: 84 inputs, 10 outputs (number of classes of EMNIST)
        else:
            self.fc3 = nn.Linear(84, 10)  # Third FC layer: 84 inputs, 10 outputs (number of classes)

    def forward(self, x):
        # Forward propagation of data through the network layers
        x = self.pool(F.relu(self.conv1(x)))  # Apply the first convolutional layer, followed by a ReLU activation function and pooling
        x = self.pool(F.relu(self.conv2(x)))  # Apply the second convolutional layer, followed by a ReLU activation function and pooling
        x = x.view(-1, 16 * 4 * 4)  # Reshape the tensor to be compatible with the fully connected layer
        x = F.relu(self.fc1(x))  # Apply the first fully connected layer, followed by a ReLU activation function
        x = F.relu(self.fc2(x))  # Apply the second fully connected layer, followed by a ReLU activation function
        x = self.fc3(x)  # Apply the third fully connected layer
        return x

In [7]:
class NetCIFAR(nn.Module):
  def __init__(self, dataset='CIFAR10'):
    super(NetCIFAR, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    if dataset == 'CIFAR100':
        self.fc3 = nn.Linear(84, 100)  # Third FC layer: 84 inputs, 100 outputs (number of classes for CIFAR100)
    elif dataset == 'CIFAR10':
        self.fc3 = nn.Linear(84, 10)  # Third FC layer: 84 inputs, 10 outputs (number of classes for CIFAR10)
    else:
        raise ValueError('Invalid dataset. Please choose one of the supported datasets CIFAR.')
        
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x


In [8]:
class NetSTL10(nn.Module):
    def __init__(self):
        super(NetSTL10, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # Adjusted to handle color images (3 input channels)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 21 * 21, 120)  # Adjusted to match the input size after pooling
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)  # Adjusted to 10 outputs for STL10 dataset

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 21 * 21)  # Adjusted to match the input size after pooling
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [9]:
def createCNN(dataset='MNIST'):
    if dataset == 'CIFAR10' or dataset == 'CIFAR100':
        net = NetCIFAR(dataset).to(device)
    elif dataset == 'STL10':
        net = NetSTL10().to(device)
    else:
        net = NetMNIST(dataset).to(device)
    
    #print('dataset CNN: ',dataset)
    #print(net)
    
    return net

In [10]:
# The method responsible for loading the training and test datasets
def load_dataset(dataset='MNIST', path='./data', batch_size=32, num_workers=1):
    # Define the transformation to be applied to the data
    transform = transforms.Compose(
    [transforms.ToTensor(),  # Convert images to tensors
     transforms.Normalize((0.5,), (0.5,))])  # Normalize images with mean 0.5 and standard deviation 0.5

    if dataset == 'MNIST':
        # Load the MNIST training dataset
        trainset = torchvision.datasets.MNIST(root=path, train=True,
                                            download=True, transform=transform)
        # Load the MNIST test dataset
        testset = torchvision.datasets.MNIST(root=path, train=False,
                                        download=True, transform=transform)
    elif dataset == 'CIFAR10':
        # Load the CIFAR10 training dataset
        trainset = torchvision.datasets.CIFAR10(root=path, train=True,
                                            download=True, transform=transform)
        # Load the CIFAR10 test dataset
        testset = torchvision.datasets.CIFAR10(root=path, train=False,
                                        download=True, transform=transform)
    elif dataset == 'CIFAR100':
        # Load the CIFAR100 training dataset
        trainset = torchvision.datasets.CIFAR100(root=path, train=True,
                                            download=True, transform=transform)
        # Load the CIFAR100 test dataset
        testset = torchvision.datasets.CIFAR100(root=path, train=False,
                                        download=True, transform=transform)
    elif dataset == 'FashionMNIST':
        # Load the FashionMNIST training dataset
        trainset = torchvision.datasets.FashionMNIST(root=path, train=True,
                                            download=True, transform=transform)
        # Load the FashionMNIST test dataset
        testset = torchvision.datasets.FashionMNIST(root=path, train=False,
                                        download=True, transform=transform)
    elif dataset == 'EMNIST':
        # Load the EMNIST training dataset
        trainset = torchvision.datasets.EMNIST(root=path, split='balanced', train=True,
                                            download=True, transform=transform)
        # Load the EMNIST test dataset
        testset = torchvision.datasets.EMNIST(root=path, split='balanced', train=False,
                                        download=True, transform=transform)
    elif dataset == 'KMNIST':
        # Load the KMNIST training dataset
        trainset = torchvision.datasets.KMNIST(root=path, train=True,
                                            download=True, transform=transform)
        # Load the KMNIST test dataset
        testset = torchvision.datasets.KMNIST(root=path, train=False,
                                        download=True, transform=transform)
    elif dataset == 'QMNIST':
        # Load the QMNIST training dataset
        trainset = torchvision.datasets.QMNIST(root=path, what='train',
                                            download=True, transform=transform)
        # Load the QMNIST test dataset
        testset = torchvision.datasets.QMNIST(root=path, what='test',
                                        download=True, transform=transform)
    elif dataset == 'STL10':
        # Load the STL10 training dataset
        trainset = torchvision.datasets.STL10(root=path, split='train',
                                            download=True, transform=transform)
        # Load the STL10 test dataset
        testset = torchvision.datasets.STL10(root=path, split='test',
                                        download=True, transform=transform)
    else:
        raise ValueError('Invalid dataset. Please choose one of the supported datasets.')

    # Create dataloaders for the training and test datasets
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=num_workers)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                            shuffle=False, num_workers=num_workers)
    
    # Return the trainloader and testloader
    return trainloader, testloader


In [11]:
training_time_list = []
testing_time_list = []
accuracy_test_list = []
loss_test_list = []
epochs = range(10)

In [12]:
#print('[torch-summary] Model Summary with torchvision.datasets.MNIST')
#summary(net, (1, 28, 28))  # Resume o modelo, fornecendo o tamanho de entrada (1 canal, 28x28 pixels)

In [13]:
def train_and_test(dataset='MNIST'):
    
    net = createCNN(dataset).to(device)  # Instancia o modelo e o move para a GPU, se disponível
    criterion = nn.CrossEntropyLoss().to(device)  # Mova a função de perda para a GPU, se disponível
    optimizer = optim.Adam(net.parameters(), lr=0.001)  # Define o otimizador para atualizar os parâmetros do modelo
    
    
    #trainloader, testloader = load_dataset(dataset)  # Load the training and test datasets
    trainloader, testloader = load_dataset(dataset)  # Load the training and test datasets


    loss_list = []
    
    print('[%s] %d training images / %d testing images' % (dataset, len(trainloader.dataset),len(testloader.dataset)))
    
    start_time = time.time()
    
    # Use tqdm to create a progress bar for the training loop
    with tqdm(total=len(trainloader)*len(epochs), unit='batch', ncols=100,desc=f"Training - {len(epochs):02d} Epochs") as pbar_training:
        for epoch in epochs:
        
            running_loss = 0.0  # Variable to store the accumulated loss during training

        
            for i, data in enumerate(trainloader, 0):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)  # Move the data to the GPU, if available

                optimizer.zero_grad()  # Zero the gradients of the parameters

                outputs = net(inputs)  # Forward pass the data through the model
                loss = criterion(outputs, labels)  # Calculate the loss

                loss.backward()  # Backpropagation to compute the gradients
                optimizer.step()  # Update the model parameters based on the gradients

                running_loss += loss.item()  # Accumulate the loss for display purposes

                if i == len(trainloader) - 1:
                    last_loss = running_loss / ((i % 100) + 1)
                    #print('[%d, %5d] Last loss: %.3f' % (epoch + 1, i+1, last_loss))
                    loss_list.append(last_loss)  # Store the epoch's average loss in the list

                if i % 100 == 99:
                    #print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
                    running_loss = 0.0

                # Update the tqdm progress bar
                pbar_training.update(1)

        end_time = time.time()
        training_time = end_time - start_time

        correct = 0  # Variable to store the number of correct predictions
        total = 0  # Variable to store the total number of test examples

        start_time = time.time()

        # Use tqdm to create a progress bar for the testing loop
    with tqdm(total=len(testloader), unit='batch', ncols=100, desc="Testing") as pbar_testing:
        with torch.no_grad():
            for data in testloader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)  # Move the data to the GPU, if available

                outputs = net(images)  # Forward pass the test data through the model
                _, predicted = torch.max(outputs.data, 1)  # Get the predictions with highest probability

                total += labels.size(0)  # Update the total number of test examples
                correct += (predicted == labels).sum().item()  # Count the number of correct predictions

                    # Update the tqdm progress bar
                pbar_testing.update(1)        

        end_time = time.time()
        testing_time = end_time - start_time

        accuracy_test = correct / total
        loss_test = (total - correct) / total

        accuracy_test_pct = 100 * accuracy_test
        loss_test_pct = 100 * loss_test

    loss_training = loss_list[-1] if loss_list else None  # Get the last element of loss_list or None if empty
    
    return {
        'loss_list': loss_list,
        'training_time': training_time,
        'testing_time': testing_time,
        'loss_training': loss_training,        
        'accuracy_test': accuracy_test,
        'loss_test': loss_test
    }


In [14]:
result_MNIST = train_and_test()
result_FashionMNIST = train_and_test('FashionMNIST')
result_EMNIST = train_and_test('EMNIST')
result_KMNIST = train_and_test('KMNIST')
result_QMNIST = train_and_test('QMNIST')
result_CIFAR10 = train_and_test('CIFAR10')
result_CIFAR100 = train_and_test('CIFAR100')
result_STL10 = train_and_test('STL10')

results = {'MNIST': result_MNIST,
           'FashionMNIST': result_FashionMNIST,
           'EMNIST':result_EMNIST,
           'KMNIST':result_KMNIST,
           'QMNIST':result_QMNIST,
           'CIFAR10':result_CIFAR10,
           'CIFAR100':result_CIFAR100,
           'STL10':result_STL10
          }

[MNIST] 60000 training images / 10000 testing images


Testing: 100%|████████████████████████████████████████████████| 313/313 [00:02<00:00, 120.06batch/s]


[FashionMNIST] 60000 training images / 10000 testing images


Testing: 100%|████████████████████████████████████████████████| 313/313 [00:02<00:00, 119.67batch/s]


[EMNIST] 112800 training images / 18800 testing images


...

Testing: 100%|████████████████████████████████████████████████| 313/313 [00:03<00:00, 101.83batch/s]


Files already downloaded and verified
Files already downloaded and verified
[CIFAR100] 50000 training images / 10000 testing images


Testing: 100%|█████████████████████████████████████████████████| 313/313 [00:03<00:00, 86.29batch/s]


Files already downloaded and verified
Files already downloaded and verified
[STL10] 5000 training images / 8000 testing images


Testing: 100%|█████████████████████████████████████████████████| 250/250 [00:08<00:00, 30.65batch/s]


In [15]:
import pandas as pd

epoch_list = [x + 1 for x in list(epochs)]

table_data = {'Epoch': epoch_list}

for dataset, result in results.items():
    loss_list = result['loss_list']
    column_name = 'Loss ' + dataset
    table_data[column_name] = loss_list

df_training = pd.DataFrame(table_data)
df_training.set_index('Epoch', inplace=True)
df_training


,Loss MNIST,Loss FashionMNIST,Loss EMNIST,Loss KMNIST,Loss QMNIST,Loss CIFAR10,Loss CIFAR100,Loss STL10
Epoch,,,,,,,,
1,0.089726,0.416776,0.519638,0.178631,0.094273,1.422449,3.577976,1.660716
2,0.047583,0.356508,0.431161,0.105822,0.049328,1.292976,3.319002,1.478024
3,0.048912,0.296615,0.408110,0.068046,0.047058,1.149686,3.141521,1.210577
4,0.047993,0.323669,0.403290,0.062656,0.042161,1.057990,3.033691,1.070730
5,0.039184,0.276312,0.335292,0.048570,0.032428,1.022587,2.909626,0.888005
6,0.022122,0.261107,0.348213,0.055086,0.025067,0.973795,2.897114,0.746448
7,0.014859,0.246660,0.397106,0.053482,0.025561,0.948463,2.794874,0.562957
8,0.021356,0.230851,0.269430,0.045760,0.016502,0.873080,2.755467,0.395778
9,0.010727,0.218875,0.296457,0.039353,0.013984,0.938096,2.706670,0.258475


In [16]:
dataset_list = []
accuracy_test_list = []
loss_test_list = []
loss_training_list = []
training_time_list = []
testing_time_list = []

# Percorrer o dicionário results e extrair os valores
for dataset, result in results.items():
    dataset_list.append(dataset)  # Adicionar o nome do dataset à lista
    
    accuracy_test = result.get('accuracy_test', None)
    if isinstance(accuracy_test, float):  # Verificar se é um número float
        accuracy_test = [accuracy_test]  # Converter para lista
    accuracy_test_list.append(accuracy_test)
    
    loss_test = result.get('loss_test', None)
    if isinstance(loss_test, float):  # Verificar se é um número float
        loss_test = [loss_test]  # Converter para lista
    loss_test_list.append(loss_test)
    
    loss_training = result.get('loss_training', None)
    if isinstance(loss_training, float):  # Verificar se é um número float
        loss_training = [loss_training]  # Converter para lista
    loss_training_list.append(loss_training)    
    
    training_time = result.get('training_time', None)
    if isinstance(training_time, float):  # Verificar se é um número float
        training_time = [training_time]  # Converter para lista
    training_time_list.append(training_time)
    
    testing_time = result.get('testing_time', None)
    if isinstance(testing_time, float):  # Verificar se é um número float
        testing_time = [testing_time]  # Converter para lista
    testing_time_list.append(testing_time)

# Create a dictionary with the list values
data = {'Dataset': dataset_list,
        'Training Time': training_time_list,
        'Testing Time': testing_time_list,
        'Loss Training': loss_training_list,
        'Loss Test': loss_test_list,
        'Accuracy Test': accuracy_test_list}

# Create a DataFrame from the dictionary
df_model_dataset = pd.DataFrame(data)

df_model_dataset.set_index('Dataset', inplace=True)

# Display the DataFrame
#df_model_dataset.transpose()
df_model_dataset


,Training Time,Testing Time,Loss Training,Loss Test,Accuracy Test
Dataset,,,,,
MNIST,[201.64672827720642],[2.611858367919922],[0.025280748233817575],[0.012],[0.988]
FashionMNIST,[205.22526931762695],[2.6216888427734375],[0.2188428450624148],[0.1015],[0.8985]
EMNIST,[383.6528193950653],[4.833750486373901],[0.29886192589998245],[0.13632978723404254],[0.8636702127659575]
KMNIST,[202.64346361160278],[2.5809340476989746],[0.030447382162092255],[0.0682],[0.9318]
QMNIST,[202.55193328857422],[15.558828353881836],[0.015559190554522501],[0.012516666666666667],[0.9874833333333334]
CIFAR10,[193.03821063041687],[3.0755178928375244],[0.843771411312951],[0.3582],[0.6418]
CIFAR100,[198.67870664596558],[3.6324987411499023],[2.6827038526535034],[0.715],[0.285]
STL10,[100.36720490455627],[8.163002252578735],[0.16246325457305238],[0.488875],[0.511125]
